### login Esker (1)

In [1]:
from selenium import webdriver # (1) login 
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver import Keys
from selenium.webdriver.support.ui import Select
import time

driver = webdriver.Chrome()
driver.get("https://az3.ondemand.esker.com/ondemand/webaccess/asf/home.aspx")
driver.maximize_window()
time.sleep(1)

driver.find_element(By.XPATH, '//*[@id="ctl03_tbUser"]').send_keys("john.tan@sh-cogent.com.sg")
driver.find_element(By.XPATH, '//*[@id="ctl03_tbPassword"]').send_keys("Esker3838")
driver.find_element(By.XPATH, '//*[@id="ctl03_btnSubmitLogin"]').click()
time.sleep(0.5)

def hover(driver, x_path):
    elem_to_hover = driver.find_element(By.XPATH, x_path)
    hover = ActionChains(driver).move_to_element(elem_to_hover)
    hover.perform()

x_path_hover = '//*[@id="mainMenuBar"]/td/table/tbody/tr/td[36]/a/div'
hover(driver, x_path_hover)
time.sleep(1)

try:
    drop_down=driver.find_element(By.XPATH, '//*[@id="DOCUMENT_TAB_100872215"]/a/div[2]').click()
    time.sleep(1)
except Exception as e:
    print(e) #VENDOR INVOICES
#driver.find_element(By.XPATH, '//*[@id="mainMenuBar"]/td/table/tbody/tr/td[36]/a/div') # 1(a)

In [2]:
def hover(driver, x_path):
    elem_to_hover = driver.find_element(By.XPATH, x_path)
    hover = ActionChains(driver).move_to_element(elem_to_hover)
    hover.perform()

x_path_hover = '//*[@id="mainMenuBar"]/td/table/tbody/tr/td[36]/a/div'
hover(driver, x_path_hover)
time.sleep(1)

try:
    drop_down=driver.find_element(By.XPATH, '//*[@id="DOCUMENT_TAB_100872215"]/a/div[2]').click()
    time.sleep(1)
except Exception as e:
    print(e) #VENDOR INVOICES
#driver.find_element(By.XPATH, '//*[@id="mainMenuBar"]/td/table/tbody/tr/td[36]/a/div') # 1(a)

### Esker (2)

In [6]:
import pandas as pd # 20240723 works good!
import numpy as np
import openpyxl

path_vendor = "https://raw.githubusercontent.com/JohnTan38/Best-README/master/"
vendor_data = pd.read_excel(path_vendor+ 'Vendor_Data.xlsx', sheet_name='Sheet1', engine='openpyxl')
vendor_payment_reference = pd.read_excel(path_vendor+ 'Vendor_Payment_Reference.xlsx', sheet_name='vendor_payment', engine='openpyxl')

vendor_data = vendor_data[['Vendor Number', 'Name1']]
vendor_payment_reference = vendor_payment_reference[['Vendor', 'Payment reference', 'Clrng doc.', 'Pstng Date']]
vendor_data.rename(columns={'Vendor Number': 'Vendor_Number'}, inplace=True)
vendor_payment_reference.rename(columns={'Payment reference': 'Payment_reference', 'Clrng doc.': 'Clrng_doc', 'Pstng Date': 'Pstng_Date'}, inplace=True)
vendor_payment_reference = vendor_payment_reference.dropna(subset=['Vendor'])

def filter_vendor_number(df):
    df['Vendor_Number'] = df['Vendor_Number'].astype(str) # Ensure 'Vendor_Number' is of string type
    # Remove rows where 'Vendor_Number' is blank, None, or NaN
    df = df[df['Vendor_Number'].astype(str).str.strip().notna() & df['Vendor_Number'].astype(str).ne('nan')]
    df = df[df['Vendor_Number'].str.startswith('1000')] # Select rows where 'Vendor_Number' starts with '1000'
    return df

def filter_payment_reference(df):
    df['Clrng_doc'] = df['Clrng_doc'].astype(str)
    df = df[df['Payment_reference'].astype(str).str.strip().notna() & df['Payment_reference'].astype(str).ne('nan')]
    df = df[df['Clrng_doc'].astype(str).str.strip().notna() & df['Clrng_doc'].astype(str).ne('nan')]
    df = df[df['Clrng_doc'].str.startswith('15')]
    return df

from datetime import datetime, timedelta
def filter_dates(df):
    df['Pstng_Date'] = pd.to_datetime(df['Pstng_Date']) # Convert 'Doc_Date' to datetime format    
    sixty_days_ago = datetime.now() - timedelta(days=60) # Get the date 60 days ago from today        
    df = df[df['Pstng_Date'] > sixty_days_ago] # Select rows where 'Doc date' is within the past 60 days from today
    return df

def filter_top_vendor(df, column_name, list_top_vendor):
    return df[df[column_name].isin(list_top_vendor)]

def get_vendor_name(top_vendor):
     return dict_vendor_data.get(str(top_vendor))

top_vendors = [1000366487,1000169276] # 1000366487 1000169276 1000152171
vendor_data = filter_vendor_number(vendor_data)
vendor_payment_reference = filter_payment_reference(vendor_payment_reference)
vendor_payment_reference_sixtydays = filter_dates(filter_payment_reference(vendor_payment_reference))

dict_vendor_data = vendor_data.set_index('Vendor_Number').to_dict()['Name1'] #%timeit

def extract_invoice_numbers(driver, x_path_invoice):
    list_invoice_numbers = [] # Initialize an empty list to store the invoice numbers
    # Get the number of rows in the table
    rows = driver.find_elements(By.XPATH, '//*[@id="tpl_ih_adminList_MainGrid"]/tbody/tr')
    # Iterate over each row
    for i in range(2, len(rows)-1):
        # Construct the XPath for the current row
        x_path_invoice_current = x_path_invoice.replace('tr[2]', f'tr[{i}]')

        # Find the element and get its innerHTML attribute
        try:
            invoice_number = driver.find_element(By.XPATH, x_path_invoice_current).get_attribute("innerHTML")
        except Exception as e:
            print(e)
        list_invoice_numbers.append(invoice_number)
    return list_invoice_numbers #get list invoice numbers on current page

def update_invoices_index(list_invoice, dict_update_vendor_payment_reference):
    result = []
    for key, value in dict_update_vendor_payment_reference.items():
        if str(key) in list_invoice:
            invoice_number_to_update = str(key)
            index_invoice_to_update = list_invoice.index(str(key))
            result.append((invoice_number_to_update, value, index_invoice_to_update))
    return result


for top_vendor in top_vendors:
    try:
        company_code = driver.find_element(By.XPATH, '//*[@id="tpl_ih_adminList_displayedFilters_ctl00_ctl02_ddl1_tagify"]/span')
        company_code.send_keys("SG77") #(2) input company code SG77
        time.sleep(0.3)
        actions = ActionChains(driver)
        num_tab = 6
        actions.send_keys(Keys.TAB*num_tab).perform()
    except Exception as e:
        print(e)
    update_vendor_payment_reference = filter_top_vendor(vendor_payment_reference_sixtydays, 'Vendor', [top_vendor])
    dict_update_vendor_payment_reference = update_vendor_payment_reference.set_index('Payment_reference').to_dict()['Clrng_doc']
    vendor_name = get_vendor_name(top_vendor)    
    try:
        vendor_name_0 = driver.find_element(By.XPATH, '//*[@id="tpl_ih_adminList_displayedFilters_ctl00_ctl03_ddl1_tagify"]/span')
        vendor_name_0.send_keys(vendor_name)
        time.sleep(0.5)
        actions.send_keys(Keys.ENTER).perform()
    except Exception as e:
        print(e)
  
    #tuple_update_payment_reference = update_invoices_index(list_invoice_numbers_on_page, dict_update_vendor_payment_reference)
    for key,value in dict_update_vendor_payment_reference.items():
        invoice_number_to_update = key
        payment_reference_to_update = value
        x_path_invoice = '//*[@id="tpl_ih_adminList_MainGrid"]/tbody/tr[2]/td[7]/a'
        list_invoice_numbers_on_page = extract_invoice_numbers(driver, x_path_invoice) #get list invoice numbers on current page
        tuple_update_payment_reference = update_invoices_index(list_invoice_numbers_on_page, dict_update_vendor_payment_reference)
        tuple_update_payment_reference = [('202402556', '1500002236.0', 5), ('202402558', '1500002236.0', 4),('72351', '1500002392.0', 2), ('2176615', '1500002310.0', 4)]#, [('202402556', '5100003086.0', 5), ('202402558', '5100003089.0', 4), ('72351', '5100002392.0', 2)] ##
 
        ## get index
        for tuple_elem in tuple_update_payment_reference:
            if str(key) == tuple_elem[0]:
                index_invoice_toClick = tuple_elem[2]
                payment_reference_to_update = tuple_elem[1].replace('.0', '')
                
                x_path_invoice_toClick = '//*[@id="tpl_ih_adminList_MainGrid"]/tbody/tr[' + str(index_invoice_toClick+2) + ']/td[7]/a'
                invoice_toClick = tuple_elem[0]
                print(invoice_toClick, x_path_invoice_toClick)
                        
                try:
                    #driver.find_element(By.XPATH, x_path_invoice_toClick).click()
                    WebDriverWait(driver,3).until(EC.element_to_be_clickable((By.LINK_TEXT, invoice_toClick))).click()
                except Exception as e:
                    print(e)
                time.sleep(5)
                try:
                    enter_payment_details=driver.find_element(By.XPATH, '//*[@id="form-footer"]/div[1]/a[5]/span').click() #Enter payment details btn
                except Exception as e:
                    print(e)
                time.sleep(3)
    
                try:
                    driver.find_element(By.XPATH, '//*[@id="Payment_pane_eskCtrlBorder_content"]/div/div/table/tbody/tr[3]/td[2]/div/input').click()
                    driver.find_element(By.XPATH, '//*[@id="Payment_pane_eskCtrlBorder_content"]/div/div/table/tbody/tr[3]/td[2]/div/input').send_keys(payment_reference_to_update)
                    time.sleep(5) #input payment reference
                except Exception as e:
                    print(e)

                try:
                    save_click = driver.find_element(By.XPATH, '//*[@id="form-footer"]/div[1]/a[1]/span')#.click() # Save btn
                except Exception as e:
                    print(e)
                #back to page 1
                try:
                    quit_click_1 = driver.find_element(By.XPATH, '//*[@id="form-footer"]/div[1]/a[2]/span').click()
                except Exception as e:
                    print(e)
                time.sleep(5)
                try:
                    quit_click_2 = driver.find_element(By.XPATH, '//*[@id="form-footer"]/div[1]/a[8]/span').click()
                except Exception as e:
                    print(e)
                time.sleep(5)
                try:
                    company_code = driver.find_element(By.XPATH, '//*[@id="tpl_ih_adminList_displayedFilters_ctl00_ctl02_ddl1_tagify"]/span')
                    company_code.send_keys("SG77") #(2) input Company code SG73, SG77, SG81, SG82, SG83, SG88
                    time.sleep(1)
                    actions = ActionChains(driver)
                    actions.send_keys(Keys.TAB*6).perform()
                    driver.find_element(By.XPATH, '//*[@id="tpl_ih_adminList_displayedFilters_ctl00_ctl03_ddl1_tagify"]/span').send_keys(vendor_name)
                    actions.send_keys(Keys.ENTER).perform()
                    time.sleep(1)
                except Exception as e:
                    print(e)
            else:
                None

202402556 //*[@id="tpl_ih_adminList_MainGrid"]/tbody/tr[7]/td[7]/a
202402558 //*[@id="tpl_ih_adminList_MainGrid"]/tbody/tr[6]/td[7]/a
72351 //*[@id="tpl_ih_adminList_MainGrid"]/tbody/tr[4]/td[7]/a


In [4]:
#dict_update_vendor_payment_reference_1 = {20242332: '1500002236.0'}

#try:
    #WebDriverWait(driver,3).until(EC.element_to_be_clickable((By.LINK_TEXT, '202402331'))).click()
    #driver.find_element(By.LINK_TEXT, '202402331').click()
#except Exception as e:
    #print(e)
dict_update_vendor_payment_reference

{72117: '1500001958.0',
 72116: '1500001958.0',
 72028: '1500001958.0',
 72351: '1500002392.0'}

In [28]:
dict_update_vendor_payment_reference_1 = {202402332: '1500002236.0'}
for key,value in dict_update_vendor_payment_reference_1.items():
        invoice_number_to_update = key
        payment_reference_to_update = value
        x_path_invoice = '//*[@id="tpl_ih_adminList_MainGrid"]/tbody/tr[2]/td[7]/a'
        list_invoice_numbers_on_page = extract_invoice_numbers(driver, x_path_invoice)
        tuple_update_payment_reference = update_invoices_index(list_invoice_numbers_on_page, dict_update_vendor_payment_reference)
        for tuple_elem in tuple_update_payment_reference:
                if str(key) == tuple_elem[0]:
                        index_invoice_toClick = tuple_elem[2]
                        payment_reference_to_update = tuple_elem[1].replace('.0', '')
                        print(index_invoice_toClick,payment_reference_to_update)
                else:
                        None
        
tuple_update_payment_reference

1 1500002236


[('202402332', '1500002236.0', 1),
 ('202402331', '1500002236.0', 0),
 ('202402406', '1500002236.0', 2)]

: 

In [ ]:
def update_invoices_index(list_invoice, dict_update_vendor_payment_reference):
    result = []
    for key, value in dict_update_vendor_payment_reference.items():
        if str(key) in list_invoice:
            invoice_number_to_update = str(key)
            index_invoice_to_update = list_invoice.index(str(key))
            result.append((invoice_number_to_update, value, index_invoice_to_update))
    return result

tuple_update_payment_reference = update_invoices_index(list_invoice_numbers_on_page, dict_update_vendor_payment_reference)

for tuple_elem in [tuple_update_payment_reference[0]]:
    x_path_invoice_toClick = '//*[@id="tpl_ih_adminList_MainGrid"]/tbody/tr[' + str(tuple_elem[2]+2) + ']/td[7]/a'
    payment_reference_to_update = tuple_elem[1].replace('.0', '')
    invoice_number_to_update = tuple_elem[0]
    print(x_path_invoice_toClick,payment_reference_to_update,invoice_number_to_update)
    try:
        driver.find_element(By.XPATH, x_path_invoice_toClick).click()
    except Exception as e:
        print(e)
    time.sleep(5)
    try:
        enter_payment_details=driver.find_element(By.XPATH, '//*[@id="form-footer"]/div[1]/a[5]/span').click() #Enter payment details
    except Exception as e:
        print(e)
    time.sleep(3)
    
    try:
        driver.find_element(By.XPATH, '//*[@id="Payment_pane_eskCtrlBorder_content"]/div/div/table/tbody/tr[3]/td[2]/div/input').click()
        driver.find_element(By.XPATH, '//*[@id="Payment_pane_eskCtrlBorder_content"]/div/div/table/tbody/tr[3]/td[2]/div/input').send_keys(payment_reference_to_update)
        time.sleep(3)
    except Exception as e:
        print(e)
    
    #back to page 1 20240719
    try:
        quit_click_1 = driver.find_element(By.XPATH, '//*[@id="form-footer"]/div[1]/a[2]/span').click()
    except Exception as e:
        print(e)
    time.sleep(5)
    try:
        quit_click_2 = driver.find_element(By.XPATH, '//*[@id="form-footer"]/div[1]/a[8]/span').click()
    except Exception as e:
        print(e)
    time.sleep(3)
    try:
        company_code = driver.find_element(By.XPATH, '//*[@id="tpl_ih_adminList_displayedFilters_ctl00_ctl02_ddl1_tagify"]/span')
        company_code.send_keys("SG77") #(2) input Company code SG73, SG77, SG81, SG82, SG83, SG88
        time.sleep(1)
        actions = ActionChains(driver)
        actions.send_keys(Keys.TAB*6).perform()
        driver.find_element(By.XPATH, '//*[@id="tpl_ih_adminList_displayedFilters_ctl00_ctl03_ddl1_tagify"]/span').send_keys(vendor_name)
        actions.send_keys(Keys.ENTER).perform()
        time.sleep(1)
    except Exception as e:
        print(e)

In [3]:
try:
    all_rows = driver.find_elements(By.XPATH, '//*[@id="tpl_ih_adminList_MainGrid"]/tbody/tr[3]/td[6]/a')
    #print(len(all_rows))
except Exception as e:
    print(e)

try:
    all_vendors = driver.find_elements(By.XPATH, '//*[@id="tpl_ih_adminList_MainGrid"]/tbody/tr[2]/td[6]/a')[0].click()
    #print(len(all_vendors))
except Exception as e:
    print(e) ##

In [51]:
#development
def get_payment_reference(df, vendor_name, invoice_number):
    # Filter the DataFrame for rows where Vendor_Name is vendor_name and Invoice_Number is invoice_number
    matching_rows = df[(df['Vendor_Name'] == vendor_name) & (df['Invoice_Number'] == invoice_number)]
    
    # If there is at least one matching row, return the Payment_Reference of the first one
    if not matching_rows.empty:
        return matching_rows['Payment_Reference'].iloc[0]
    else:
        return None

list_vendor =[]
list_invoice =[]
list_payment_reference =[]
pg =0 # 20240717 works!
next_page_disable = driver.find_elements(By.XPATH, '//*[@class="LinkButton LinkButton_Disabled AdminList_Table_NextPage"]')#.get_property('disabled')

while len(next_page_disable) ==0:
    for i in range(0, 20): #len(all_rows)+1
        try:
            x_path_vendor = '//*[@id="tpl_ih_adminList_MainGrid"]/tbody/tr[' + str(i+2) + ']/td[6]/a'
            x_path_invoice = '//*[@id="tpl_ih_adminList_MainGrid"]/tbody/tr[' + str(i+2) + ']/td[7]/a'
    
            vendor_name = driver.find_elements(By.XPATH, x_path_vendor)[0].get_attribute("innerHTML")
            invoice_number = driver.find_elements(By.XPATH, x_path_invoice)[0].get_attribute("innerHTML")
            vendor_name = vendor_name.replace('amp;', '')
            list_vendor.append(vendor_name)
            list_invoice.append(invoice_number)
    
            payment_reference = get_payment_reference(payment_ref, vendor_name, invoice_number)
            if payment_reference is not None:
                list_payment_reference.append(payment_reference)
                
            else:
                list_payment_reference.append('')
                continue
        except Exception as e:
            print(e)
    try:
        driver.find_element(By.XPATH, '//*[@id="tpl_ih_adminList_NextPageLinkButtonTopBar"]').click()
    except Exception as e:
        print(e) #click to next page
    pg +=1
    next_page_disable = driver.find_elements(By.XPATH, '//*[@class="LinkButton LinkButton_Disabled AdminList_Table_NextPage"]')
print(pg)

if next_page_disable is not None:
    list_vendor_lastpage =[]
    list_invoice_lastpage =[]
    list_payment_reference_lastpage =[]
    for i in range(0, 20):
        try:
            x_path_vendor = '//*[@id="tpl_ih_adminList_MainGrid"]/tbody/tr[' + str(i+2) + ']/td[6]/a'
            x_path_invoice = '//*[@id="tpl_ih_adminList_MainGrid"]/tbody/tr[' + str(i+2) + ']/td[7]/a'
    
            vendor_name = driver.find_elements(By.XPATH, x_path_vendor)[0].get_attribute("innerHTML")
            invoice_number = driver.find_elements(By.XPATH, x_path_invoice)[0].get_attribute("innerHTML")
            vendor_name = vendor_name.replace('amp;', '')
            list_vendor_lastpage.append(vendor_name)
            list_invoice_lastpage.append(invoice_number)

            payment_reference = get_payment_reference(payment_ref, vendor_name, invoice_number)
            if payment_reference is not None:
                list_payment_reference_lastpage.append(payment_reference)
                #print(payment_reference)
            else:
                list_payment_reference_lastpage.append('')
                continue
        except Exception as e:
            print(e)

#vendor_name = driver.find_elements(By.XPATH, '//*[@id="tpl_ih_adminList_MainGrid"]/tbody/tr[2]/td[6]/a')[0].get_attribute("innerHTML")
#invoice_number = driver.find_elements(By.XPATH, '//*[@id="tpl_ih_adminList_MainGrid"]/tbody/tr[2]/td[7]/a')[0].get_attribute("innerHTML") 

1
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range


### Esker (3)

In [ ]:
list_vendor_all = list_vendor + list_vendor_lastpage # (3) 20240717 works!
list_invoice_all = list_invoice + list_invoice_lastpage
list_payment_reference_all = list_payment_reference + list_payment_reference_lastpage

dict_invoice_payment_reference = {z[0]: list(z[1:]) for z in zip(list_invoice_all, list_vendor_all, list_payment_reference_all)}
#dict_invoice_payment_reference

def extract_values_from_invoices(dictionary_invoice, list_invoice):
    results = [] #20240718

    for invoice in list_invoice:
        if invoice in dictionary_invoice:
            vendor_name, payment_reference = dictionary_invoice[invoice]
            results.append((vendor_name, payment_reference, list_invoice.index(invoice)))
    return results
tuple_payment_reference = extract_values_from_invoices(dict_invoice_payment_reference, list_invoice_all)
#tuple_payment_reference

def get_indices(tuple_payment_reference, payment_reference_to_update):
    index_payment_reference = [] #20240718
    for payment_reference in payment_reference_to_update:
        for tuple_element in tuple_payment_reference:
            if tuple_element[1] == payment_reference:
                index_payment_reference.append([tuple_element[1],tuple_element[2]])
    return index_payment_reference

index_payment_reference = get_indices(tuple_payment_reference, payment_reference_to_update)
#index_payment_reference

index_payment_reference_1 = [['INSPRO-3269', 21], ['RESOLUTE-6890', 4]] #20240718
for i in range(len(index_payment_reference_1)):
    value = index_payment_reference_1[i][1]
    if value / 20 < 1:
        x_path_invoice_toClick = '//*[@id="tpl_ih_adminList_MainGrid"]/tbody/tr[' + str(value+2) + ']/td[7]/a'
        driver.find_element(By.XPATH, x_path_invoice_toClick).click()
    else:
        pg_indx = value // 20
        try:
            for _ in range(pg_indx):
                    driver.find_element(By.XPATH, '//*[@id="tpl_ih_adminList_NextPageLinkButtonTopBar"]').click() #click multiple times    
        except Exception as e:
            print(e) #click to next page
        x_path_invoice_toClick = '//*[@id="tpl_ih_adminList_MainGrid"]/tbody/tr[' + str((value-20*pg_indx)+2) + ']/td[7]/a'
        driver.find_element(By.XPATH, x_path_invoice_toClick).click()

    #enter payment details
    time.sleep(3)
    try:
        enter_payment_details=driver.find_element(By.XPATH, '//*[@id="form-footer"]/div[1]/a[5]/span').click()
    except Exception as e:
        print(e)
    time.sleep(5)
    
    try:
        driver.find_element(By.XPATH, '//*[@id="Payment_pane_eskCtrlBorder_content"]/div/div/table/tbody/tr[3]/td[2]/div/input').click()
        driver.find_element(By.XPATH, '//*[@id="Payment_pane_eskCtrlBorder_content"]/div/div/table/tbody/tr[3]/td[2]/div/input').send_keys(index_payment_reference_1[i][0])
      
    except Exception as e:
        print(e)
    
    try:
        save_click = driver.find_element(By.XPATH, '//*[@id="form-footer"]/div[1]/a[1]/span')#.click() # Save
    except Exception as e:
        print(e)
    time.sleep(1)

    #back to page 1 20240719
    try:
        quit_click_1 = driver.find_element(By.XPATH, '//*[@id="form-footer"]/div[1]/a[2]/span').click()
    except Exception as e:
        print(e)
    time.sleep(5)
    try:
        quit_click_2 = driver.find_element(By.XPATH, '//*[@id="form-footer"]/div[1]/a[8]/span').click()
    except Exception as e:
        print(e)
    time.sleep(1)
    try:
        company_code = driver.find_element(By.XPATH, '//*[@id="tpl_ih_adminList_displayedFilters_ctl00_ctl02_ddl1_tagify"]/span')
        company_code.send_keys("SG80") #(2) input Company code SG73, SG77, SG81, SG82, SG83, SG88
        time.sleep(0.3)
        actions = ActionChains(driver)
        actions.send_keys(Keys.ENTER).perform() 
    except Exception as e:
        print(e)

In [35]:
def extract_values_from_invoices(dictionary_invoice, list_invoice):
    results = [] #20240718

    for invoice in list_invoice:
        if invoice in dictionary_invoice:
            vendor_name, payment_reference = dictionary_invoice[invoice]
            results.append((vendor_name, payment_reference, list_invoice.index(invoice)))
    return results
tuple_payment_reference = extract_values_from_invoices(dict_invoice_payment_reference, list_invoice_all)
tuple_payment_reference

[('DEEEPLABS PTE. LTD.', '', 0),
 ('MSIG INSURANCE (S) PTE LTD', '', 1),
 ('GREAT AMERICAN INSURANCE COMPANY', '', 2),
 ('KIM SOON LEE LOGISTICS (S) PTE. LTD', '', 3),
 ('RESOLUTE SOLUTIONS PTE LTD', 'RESOLUTE-6890', 4),
 ('3B EXPRESS SERVICES PTE. LTD.', '', 5),
 ('H-A LOGISTICS CO., LTD.', '', 6),
 ('AZON TRANSPORT PTE. LTD.', '', 7),
 ('KANG LOGISTICS PTE LTD', '', 8),
 ('KANG LOGISTICS PTE LTD', '', 9),
 ('KANG LOGISTICS PTE LTD', '', 10),
 ('KANG LOGISTICS PTE LTD', '', 11),
 ('H-A LOGISTICS CO., LTD.', '', 12),
 ('OCTA GLOBAL PTE. LTD.', '', 13),
 ('OCTA GLOBAL PTE. LTD.', '', 14),
 ('H-A LOGISTICS CO., LTD.', '', 15),
 ('H-A LOGISTICS CO., LTD.', '', 16),
 ('EVER GLORY LOGISTICS PTE LTD', '', 17),
 ('FAMOUS AIR & SEA SERVICES PTE. LTD.', '', 18),
 ('AZON TRANSPORT PTE. LTD.', '', 19),
 ('OCTA GLOBAL PTE. LTD.', 'OCTA-3269', 20),
 ('INSPRO INSURANCE BROKERS PTE LTD', '', 21),
 ('CITY CONTAINER (S) PTE LTD', '', 22),
 ('CITY CONTAINER (S) PTE LTD', '', 23),
 ('CITY CONTAINER (S) P

In [37]:
payment_reference_to_update

['RESOLUTE-6890', 'OCTA-3269']

In [38]:
def get_indices(tuple_payment_reference, payment_reference_to_update):
    index_payment_reference = [] #20240718
    for payment_reference in payment_reference_to_update:
        for tuple_element in tuple_payment_reference:
            if tuple_element[1] == payment_reference:
                index_payment_reference.append([tuple_element[1],tuple_element[2]])
    return index_payment_reference

index_payment_reference = get_indices(tuple_payment_reference, payment_reference_to_update)
index_payment_reference

[['RESOLUTE-6890', 4], ['OCTA-3269', 20]]

In [ ]:
index_payment_reference_1 = [['HERCULEAN-3269', 41]] #20240718
for i in range(len(index_payment_reference_1)):
    value = index_payment_reference_1[i][1]
    if value / 20 < 1:
        x_path_invoice_toClick = '//*[@id="tpl_ih_adminList_MainGrid"]/tbody/tr[' + str(value+2) + ']/td[7]/a'
        driver.find_element(By.XPATH, x_path_invoice_toClick).click()
    else:
        pg_indx = value // 20
        try:
            for _ in range(pg_indx):
                    driver.find_element(By.XPATH, '//*[@id="tpl_ih_adminList_NextPageLinkButtonTopBar"]').click() #click multiple times    
        except Exception as e:
            print(e) #click to next page
        x_path_invoice_toClick = '//*[@id="tpl_ih_adminList_MainGrid"]/tbody/tr[' + str((value-20*pg_indx)+2) + ']/td[7]/a'
        driver.find_element(By.XPATH, x_path_invoice_toClick).click()

    #enter payment details
    try:
        enter_payment_details=driver.find_element(By.XPATH, '//*[@id="form-footer"]/div[1]/a[5]/span').click()
    except Exception as e:
        print(e)
    time.sleep(3)
    
    try:
        driver.find_element(By.XPATH, '//*[@id="Payment_pane_eskCtrlBorder_content"]/div/div/table/tbody/tr[3]/td[2]/div/input').click()
        driver.find_element(By.XPATH, '//*[@id="Payment_pane_eskCtrlBorder_content"]/div/div/table/tbody/tr[3]/td[2]/div/input').send_keys(index_payment_reference_1[i][0])
      
    except Exception as e:
        print(e)
    
    try:
        save_click = driver.find_element(By.XPATH, '//*[@id="form-footer"]/div[1]/a[1]/span')#.click() #click Save
    except Exception as e:
        print(e)

In [41]:
def calculate_values(list_index_payment):
    # Iterate over each element in the list
    for i in range(len(list_index_payment)):
        # Extract the value at index 1
        value = list_index_payment[i][1]
        
        # Check the conditions and update the value accordingly
        if value / 20 < 1:
            # If value[1]/20 < 1, remain the same value[1]
            continue
        else:
            # If value[1]/20 > 1, then return the numerator rounded down as integer
            list_index_payment[i][1] = value // 20

    return list_index_payment
calculate_values(index_payment_reference)

[['RESOLUTE-6890', 4], ['OCTA-3269', 1]]

In [44]:
index_payment_reference_1 = [['HERCULEAN-3269', 41]]
for i in range(len(index_payment_reference_1)):
    value = index_payment_reference_1[i][1]
    if value / 20 < 1:
        x_path_invoice_toClick = '//*[@id="tpl_ih_adminList_MainGrid"]/tbody/tr[' + str(value+2) + ']/td[7]/a'
        driver.find_element(By.XPATH, x_path_invoice_toClick).click()
    else:
        pg_indx = value // 20
        try:
            for _ in range(pg_indx):
                    driver.find_element(By.XPATH, '//*[@id="tpl_ih_adminList_NextPageLinkButtonTopBar"]').click() #click multiple times    
        except Exception as e:
            print(e) #click to next page
        x_path_invoice_toClick = '//*[@id="tpl_ih_adminList_MainGrid"]/tbody/tr[' + str((value-20*pg_indx)+2) + ']/td[7]/a'
        driver.find_element(By.XPATH, x_path_invoice_toClick).click()


In [37]:
def extract_invoice_numbers(driver, x_path_invoice):
    list_invoice_numbers = [] # Initialize an empty list to store the invoice numbers
    # Get the number of rows in the table
    rows = driver.find_elements(By.XPATH, '//*[@id="tpl_ih_adminList_MainGrid"]/tbody/tr')
    # Iterate over each row
    for i in range(0, len(rows)-1):
        # Construct the XPath for the current row
        x_path_invoice_current = '//*[@id="tpl_ih_adminList_MainGrid"]/tbody/tr['+str(i+2)+']/td[7]/a'

        # Find the element and get its innerHTML attribute
        invoice_number = driver.find_element(By.XPATH, x_path_invoice_current).get_attribute("innerHTML")
        print(invoice_number)
        list_invoice_numbers.append(invoice_number)
    return list_invoice_numbers

#x_path_invoice = '//*[@id="tpl_ih_adminList_MainGrid"]/tbody/tr[2]/td[7]/a'
extract_invoice_numbers(driver, x_path_invoice)

202402331
202402332
202402406
202402557
202402558
202402556
202402614
202402629
202402630
202402673
202402927
202402928
202403011
202403114
202403113
202403170
202403171
202403312
202403280
202403265


['202402331',
 '202402332',
 '202402406',
 '202402557',
 '202402558',
 '202402556',
 '202402614',
 '202402629',
 '202402630',
 '202402673',
 '202402927',
 '202402928',
 '202403011',
 '202403114',
 '202403113',
 '202403170',
 '202403171',
 '202403312',
 '202403280',
 '202403265']

In [31]:
def find_not_updated_payment_references(dictionary_invoice, tuple_payment_reference):
    not_updated_payment_reference = []
    payment_references_set = set(pr for _, pr, _ in tuple_payment_reference)

    for invoice, (_, payment_reference) in dictionary_invoice.items():
        if payment_reference not in payment_references_set:
            not_updated_payment_reference.append(payment_reference)

    return not_updated_payment_reference
find_not_updated_payment_references(dict_invoice_payment_reference, extract_values_from_invoices(dict_invoice_payment_reference, list_invoice_all))